In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

train = pd.read_csv('tr.csv')
train.drop('label',axis = 1)
df = pd.DataFrame(train)
df2 = df.drop('label',axis=1)
df2 = df2/255.0
df2 = df2.values.reshape(-1,28,28,1)





In [4]:
test = pd.read_csv('test.csv')
df1 = pd.DataFrame(test)
df3 = df['label']
df1 = df1.values.reshape(-1,28,28,1)





In [5]:
y_train=to_categorical(df3,num_classes=10)


In [6]:

x_train,x_test,y_train, y_test = train_test_split(df2,y_train,test_size=0.1,random_state=42) 


In [7]:
x_train.shape

(37800, 28, 28, 1)

In [8]:

y_train.shape


(37800, 10)

In [9]:
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

model = Sequential()
  
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

    
model.add(Flatten()),
model.add(Dense(256, activation="relu")),
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])




In [10]:
gen_img = ImageDataGenerator(rotation_range=10, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.1, 
                             zoom_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False,
                             fill_mode='nearest'
                            )
gen_img.fit(x_train)


In [11]:
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=2)


In [ ]:
model.fit_generator(gen_img.flow(x_train, y_train), epochs=8,validation_data=(x_test,y_test),callbacks=[early_stop])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/8
 359/1182 [========>.....................] - ETA: 5:43 - loss: 0.7351 - accuracy: 0.7564

In [ ]:
loss_val, acc_val = model.evaluate(x_test,y_test)
print(loss_val, acc_val)

In [ ]:
model.save('num_reader.model')

In [ ]:
new_model = tf.keras.models.load_model('num_reader.model')

In [ ]:
predictions = new_model.predict(x_test)

In [ ]:
print(predictions)


In [ ]:
results = model.predict(df1)
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")


In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("dig_recog_sub.csv",index=False)
submission



In [ ]:
type(results)
